In [17]:
import pandas as pd
import os
import configparser
import itertools
import sqlite3
import time
import random
import locale
import logging
from logging.config import fileConfig
import traceback

In [2]:
fileConfig('config/logging_config.ini')
logger = logging.getLogger()

In [3]:
config = configparser.ConfigParser()
config.read('config/system.ini')

['config/system.ini']

In [4]:
raw_data_dir = config['path'].get('data_dir')
db_path = config['path'].get('db_path')
table = config['db'].get('table')
wait_state = config['db'].get('wait_state')
processing_state = config['db'].get('processing_state')
success_state = config['db'].get('success_state')
log_path = config['log']['log_path']

# get raw_data

In [ ]:
data = pd.read_excel('data/full_讲题_10000_-1_0301.xlsx')

In [ ]:
def get_text(df):
    line_list = []
    for index,row in df.iterrows():
        line = '**{}** : {}  '.format(row['who'],row['text'])
        line_list.append(line)
    return line_list

In [ ]:
for index,group in data.groupby('段index'):
    line_list = get_text(group)
    output_path = os.path.join(raw_data_dir,'{}.txt'.format(int(index)))
    with open(output_path,'w') as f:
        f.write('\n'.join(line_list))

# 标注数据准备

In [6]:
num_raw_data = len(os.listdir(raw_data_dir))

In [7]:
pair_list = list(itertools.combinations(range(num_raw_data),2))
random.seed(2019)
random.shuffle(pair_list)

# sqlite3

In [8]:
def loadText(index):
    path = os.path.join(raw_data_dir, '{}.txt'.format(int(index)))
    return open(path, 'r').read()


def getNowTime():
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [9]:
if os.path.exists(db_path):
    logger.info('Remove old db,path is {}'.format(db_path))
    os.remove(db_path)
conn = sqlite3.connect(db_path)
c = conn.cursor()
logger.info('Create table,name is {}'.format(table))
# Create table
c.execute('''CREATE TABLE {}
             (id,index_i, index_j,label,state,create_time,label_time)'''.
          format(table))

In [10]:
logger.info('Write pairs to table {}'.format(table))
for id, pair in enumerate(pair_list):
    create_time = getNowTime()
    c.execute("INSERT INTO pairs VALUES ({},{},{},{},{},'{}','{}')".format(
        id + 1, pair[0], pair[1], -1, wait_state, create_time, ''))
conn.commit()
conn.close()
logger.info('Finish create table {}'.format(table))

In [11]:
class myMysql:
    def __init__(self, dbhost, dbport, dbname, dbuser, dbpassword):
        self.dbhost = dbhost
        self.dbport = dbport
        self.dbname = dbname
        self.dbuser = dbuser
        self.dbpassword = dbpassword

    def connect(self):
        self.connection = pymysql.connect(host=self.dbhost,
                                          user=self.dbuser,
                                          port=self.dbport,
                                          password=self.dbpassword,
                                          db=self.dbname,
                                          charset='utf8',
                                          cursorclass=pymysql.cursors.DictCursor)
        self.cursor = self.connection.cursor()
        self.connection.autocommit(True)

    def insert(self, table_name, data):
        self.connect()
        items = list(data.keys())
        keys = ",".join(items)
        valueList = list(data.values())
        values = str(valueList)[1:-1]
        sql = "replace into {} ({}) values ({})".format(
            table_name, keys, values)
        self.cursor.execute(sql)
        self.close()

    def execute(self, sql):
        self.connect()
        self.cursor.execute(sql)
        result = self.cursor.fetchall()
        self.close()
        return result

    def close(self):
        self.connection.close()

In [18]:
class database():
    def __init__(self):
        pass
    def connect(self):
        self.conn = sqlite3.connect(db_path)
        self.conn.row_factory = sqlite3.Row
        self.cursor = self.conn.cursor()
    def execute(self, sql):
        self.connect()
        try:
            self.cursor.execute(sql)
            self.conn.commit()
            result = self.cursor.fetchall()
            result = [dict(x) for x in result]
            self.conn.close()
        except:
            logger.Error('Write to db fail,detail is {}'.format(traceback.format_exc()))
        finally:
            self.conn.close()
        return result

In [19]:
class Pair():
    def __init__(self):
        self.state = True
        sql = "select * from pairs where state==0 limit 1"
        self.result = self.execute(sql)
        if len(self.result) == 0:
            self.state = False
            logger.info('All data have been labeled ~~')
        if self.state:
            logger.info('Start select one pair')
            self.result = self.result[0]
            self.i = self.result['index_i']
            self.j = self.result['index_j']
            self.id = self.result['id']
            sql = 'UPDATE {} SET state = {} WHERE id = {}'.format(
                table, processing_state, self.id)
            self.execute(sql)
            logger.info('Pair info i={},j={},id={}'.format(
                self.i, self.j, self.id))

    def execute(self, sql):
        db = database()
        result = db.execute(sql)
        return result

    def save(self, label):
        logger.info('Save result,id = {},label = {}'.format(self.id, label))
        sql = 'UPDATE {} SET label = {},state = {},label_time = "{}" WHERE id = {}'.format(
            table, label, success_state, getNowTime(), self.id)
        self.execute(sql)

    def relase(self):
        logger.info('Relase pair(id = {})'.format(self.id))
        sql = 'UPDATE {} SET state = {} WHERE id = {}'.format(
            table, wait_state, self.id)
        self.execute(sql)

    def get_index(self):
        return self.i, self.j

    def get_text(self):
        return loadText(self.i), loadText(self.j)

state
- 0 未标注 
- 1 正在标注
- 2 标注完成

In [16]:
%%time
test_pair = Pair()
test_pair.save(1)

CPU times: user 9.39 ms, sys: 368 µs, total: 9.76 ms
Wall time: 191 ms


In [ ]:
result = c.execute("select * from pairs where state == 0 limit 10")

In [ ]:
c.execute('UPDATE {} SET label =  WHERE id = {}'.format(table,1,1))
result = conn.

In [ ]:
# result = c.execute("select * from pairs where label= -1 and state==0 limit 10")
result = c.execute('UPDATE {} SET label ={}  WHERE id = {}'.format(table,1,1))
result = [dict(x) for x in result]
pd.DataFrame(result)

In [ ]:
result